# Instalação e Pré Requisitios

Instalar Google Chrome. _Rodar Manualmente em WSL_

``` bash
wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list
sudo apt update && sudo apt install -y google-chrome-stable
```

Instalar depenências utilizando `-venv`

``` bash
# configura ambiente virtual
python3 -m venv venv

# ativa ambiente virtual
source venv/bin/activate

# instalar dependências 
pip install -r requirements.txt
```


# Image Scapper

In [1]:
from typing import List, Iterator

Utilitário para exibir imagens

In [2]:
from IPython.display import clear_output, HTML

def display_images_grid(images: Iterator[str]):
    clear_output(wait=True)

    display(HTML(
        ''.join([
            f'<img src="{img}" style="max-width: 150px; margin: 5px; display: inline-block;">'
            for img in images
        ])
    ))


Realiza o download das imagens

In [ ]:
root:str = "Dataset"
!rm -rf {root}

num_images:int = 200
query_categorias:List[str] = ["Spider Man", "Batman"]
variations = ["Happy", "Neutral", "Old", "Timeless", "Surprise"] # Quanto a quantidade de imagens não é suficiente.

images: List[str] = []
def on_save(file_path: str):
    images.append(file_path)
    display_images_grid(reversed(images[-8:]))

import web_scapper
for query in query_categorias:
    web_scapper.scrape_images(query, num_images=num_images, main_folder=root, on_save=on_save, variations=variations)

# Transfer Learning

In [ ]:
import os

#if using Theano with GPU
os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [ ]:

train_split, val_split = 0.7, 0.15
categories = [x[0] for x in os.walk(root) if x[0]][1:]

print(categories)

In [6]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [7]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [8]:
random.shuffle(data)

In [9]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

In [ ]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

In [ ]:
# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)

In [14]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

In [15]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=12,
                         epochs=10, 
                         validation_data=(x_val, y_val))

## Salvar e Carregar Modelo

In [17]:
# Método 1: Salvar o modelo completo
model_new.save('modelo_categorias.keras')


In [18]:
from keras.models import load_model

if "model_new" not in locals():
    model_new = load_model('modelo_categorias.keras')


# Teste do Modelo

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_probabilities(probs):
  cat = list(map(lambda x: x.replace(root + "/", ""), categories))

  joined = list(zip(cat, probs))
  joined.sort(key=lambda x: x[1], reverse=True) # sort by the highest probability
  return joined


# Create a file upload widget
upload_widget = widgets.FileUpload(
    accept='image/*',  # Accept all file types
    multiple=True      # Allow only one file at a time
)

display(upload_widget)

def handle_upload(_):
    
    clear_output(wait=True)
    display(upload_widget)

    for up in upload_widget.value:
        uploaded_file = up  
        content = uploaded_file['content']
        name = uploaded_file['name']

        # salvar imagem no diretorio
        with open(name, 'wb') as f:
            f.write(content)


        img, x = get_image(name)
        probabilities = model_new.predict([x])

        display(img, name, get_probabilities(probabilities[0]))


# Attach the handler to the widget
upload_widget.observe(handle_upload)